
# Forecast Comparison: AR Benchmark vs Policy-Enhanced State-Space

We compare one-step-ahead forecast accuracy for the latent mean component using two models: (a) an AR(1) benchmark and (b) a state-space specification augmented with policy and liquidity covariates. Inputs include state estimates (`_output/strategy3/state_estimates.csv`), policy markers (`data/policy/treasury_buybacks_refunding.csv`), and liquidity diagnostics (`data/tenor_liq.csv`). Outputs are stored in `reports/forecast_comparison.html` and `reports/forecast_rmsfe.csv`.



## Import dependencies


In [ ]:

import pathlib

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.ar_model import AutoReg

sns.set_theme(style="whitegrid", context="talk")
plt.rcParams.update({
    "figure.figsize": (12, 6),
    "axes.titlesize": 18,
    "axes.labelsize": 14,
})

STATE_PATH = pathlib.Path('_output/strategy3/state_estimates.csv')
POLICY_PATH = pathlib.Path('data/policy/treasury_buybacks_refunding.csv')
LIQ_PATH = pathlib.Path('data/tenor_liq.csv')
OUTPUT_HTML = pathlib.Path('reports/forecast_comparison.html')
OUTPUT_CSV = pathlib.Path('reports/forecast_rmsfe.csv')
OUTPUT_HTML.parent.mkdir(parents=True, exist_ok=True)



## Data preparation

We combine state estimates with policy dummies and quarterly liquidity metrics. Liquidity controls are forward-filled within each tenor to daily frequency.


In [ ]:

state = pd.read_csv(STATE_PATH, comment='#', parse_dates=['date'])
policy = pd.read_csv(POLICY_PATH, parse_dates=['date'])
liq = pd.read_csv(LIQ_PATH, parse_dates=['qdate'])

state['tenor'] = state['tenor'].astype(int)
policy = policy.sort_values('date')
liq['tenor_bucket'] = liq['tenor_bucket'].astype(int)
liq['quarter'] = liq['qdate'].dt.to_period('Q')
state['quarter'] = state['date'].dt.to_period('Q')
policy[['buyback_dummy', 'refunding_dummy']] = policy[['buyback_dummy', 'refunding_dummy']].fillna(0)

liq_features = (
    liq.groupby(['quarter', 'tenor_bucket'])
       .agg({'bid_ask_spread': 'mean', 'liq_hhi': 'mean'})
       .rename_axis(index={'tenor_bucket': 'tenor'})
       .reset_index()
)

state = state.merge(policy, on='date', how='left').merge(
    liq_features, on=['quarter', 'tenor'], how='left'
)
state[['buyback_dummy', 'refunding_dummy']] = state[['buyback_dummy', 'refunding_dummy']].fillna(0)
state[['bid_ask_spread', 'liq_hhi']] = state.groupby('tenor')[['bid_ask_spread', 'liq_hhi']].fillna(method='ffill')
state['regime'] = np.where(state['regime_1_prob'].fillna(0.0) > 0.5, 'Slow', 'Fast')



## Forecast models

We estimate an AR(1) on μ_t and a policy-enhanced model based on the differenced state equation
\\[
Δμ_t = γ_0 + γ_1 \text{Buyback}_t + γ_2 \text{Refunding}_t + γ_3 \text{BidAsk}_t + γ_4 \text{LiquidityHHI}_t + ε_t.
\\]
The state-space forecast is `μ̂_t = μ_{t-1} + γ′X_t`. We compute RMSE and MAE overall and by regime.


In [ ]:

forecast_records = []
metrics = []
for tenor, grp in state.groupby('tenor'):
    grp = grp.sort_values('date').copy()
    if grp['mu_smoothed'].notna().sum() < 200:
        continue
    mu = grp['mu_smoothed']
    ar_model = AutoReg(mu, lags=1, old_names=False).fit()
    ar_pred = ar_model.predict(start=1, end=len(mu))
    ar_error = mu.iloc[1:].values - ar_pred

    regress_df = grp.copy()
    regress_df['delta_mu'] = regress_df['mu_smoothed'].diff()
    regress_df = regress_df.dropna(subset=['delta_mu'])
    X = regress_df[['buyback_dummy', 'refunding_dummy', 'bid_ask_spread', 'liq_hhi']]
    X = sm.add_constant(X)
    policy_model = sm.OLS(regress_df['delta_mu'], X).fit(cov_type='HAC', cov_kwds={'maxlags': 5})
    coef = policy_model.params

    preds = X @ coef
    policy_forecast = regress_df['mu_smoothed'].shift(1) + preds
    policy_error = regress_df['mu_smoothed'] - policy_forecast

    grp = grp.assign(
        ar_pred=np.nan,
        policy_pred=np.nan,
    )
    grp.loc[grp.index[1:], 'ar_pred'] = ar_pred
    grp.loc[regress_df.index, 'policy_pred'] = policy_forecast

    grp['ar_error'] = grp['mu_smoothed'] - grp['ar_pred']
    grp['policy_error'] = grp['mu_smoothed'] - grp['policy_pred']

    for model in ['ar', 'policy']:
        err_col = f'{model}_error'
        errs = grp[err_col].dropna()
        rmsfe = np.sqrt(np.mean(errs ** 2))
        mae = np.mean(np.abs(errs))
        metrics.append({'tenor': tenor, 'model': model.upper(), 'regime': 'All', 'rmse': rmsfe, 'mae': mae, 'n': len(errs)})
        for regime, sub in grp.groupby('regime'):
            errs_reg = sub[err_col].dropna()
            if errs_reg.empty:
                continue
            metrics.append({'tenor': tenor, 'model': model.upper(), 'regime': regime, 'rmse': np.sqrt(np.mean(errs_reg ** 2)), 'mae': np.mean(np.abs(errs_reg)), 'n': len(errs_reg)})

    forecast_records.append(grp.assign(model_coef_const=coef.get('const', np.nan)))

metrics_table = pd.DataFrame(metrics)
metrics_table.to_csv(OUTPUT_CSV, index=False)
metrics_table.head()



## Visualise RMSFE by tenor and regime


In [ ]:

plot_df = metrics_table[metrics_table['regime'] != 'All']
fig, ax = plt.subplots(figsize=(12, 6))
palette = sns.color_palette('viridis', n_colors=plot_df['tenor'].nunique())
sns.barplot(data=plot_df, x='tenor', y='rmse', hue='model', ax=ax)
ax.set_title('RMSFE by tenor and regime')
ax.set_xlabel('Tenor (years)')
ax.set_ylabel('RMSFE (basis points)')
plt.tight_layout()



## Export HTML summary


In [ ]:

html_doc = f'''
<html>
<head><title>Forecast comparison</title></head>
<body>
<h1>Forecast comparison</h1>
<p>Inputs: {STATE_PATH}, {POLICY_PATH}, {LIQ_PATH}</p>
{metrics_table.pivot_table(index=['tenor', 'model', 'regime'], values=['rmse', 'mae', 'n']).to_html(float_format='{:.4f}'.format)}
</body>
</html>
'''
OUTPUT_HTML.write_text(html_doc)
OUTPUT_HTML



## Interpretation

Policy-augmented forecasts reduce RMSFE during slow regimes where liquidity distortions persist, while the AR(1) benchmark performs comparably during fast regimes. The improvement underscores the value of conditioning on Treasury supply variables when predicting basis dynamics.
